In [22]:
import re

In [23]:
with open('data/AntiScam_annotated.txt') as f:
    lines = f.readlines()

In [24]:
lines

['0\tHello my name is NAMESLOTA and I am with Amazon customer support.\tgreeting\tname\t\t\t\t\t\t\t\t\t\t\n',
 '1\tHello NAMESLOTA.\t\t\tgreeting\tname\t\t\t\t\t\t\t\t\n',
 '1\tI recently purchased a heater but it has not arrived\t\t\tnonresponsive_statement\torder_detail\t\t\t\t\t\t\t\t\n',
 "0\tI'm sorry to hear that.\tapology\torder_detail\t\t\t\t\t\t\t\t\t\t\n",
 '0\tCan I have your name and address to look this up?\telicitation\taddress\t\t\t\t\t\t\t\t\t\t\n',
 '1\tWhy is my address necessary NAMESLOTA?\t\t\topen_question\taddress\t\t\t\t\t\t\t\t\n',
 '0\t"Due to our volume, we have certain guidelines on information needed for order inquiries."\tresponsive_statement\taddress\t\t\t\t\t\t\t\t\t\t\n',
 '1\tAlright before I proceed may I have your full name for personal record.\t\t\telicitation\tname\t\t\t\t\t\t\t\t\n',
 "0\tI'm sorry that is against our policies.\trefusal\tname\t\t\t\t\t\t\t\t\t\t\n",
 '0\tI can give you my employee number if you need.\tnonresponsive_statement\tiden

In [25]:
conversations = [[]]
for line in lines:
    if line.strip() == "":
        conversations.append([])
    else:
        conversations[-1].append(re.sub(r"[\t]+", "\t", line.strip().replace("\"", "")).split("\t"))

In [26]:
conversations

[[['0',
   'Hello my name is NAMESLOTA and I am with Amazon customer support.',
   'greeting',
   'name'],
  ['1', 'Hello NAMESLOTA.', 'greeting', 'name'],
  ['1',
   'I recently purchased a heater but it has not arrived',
   'nonresponsive_statement',
   'order_detail'],
  ['0', "I'm sorry to hear that.", 'apology', 'order_detail'],
  ['0',
   'Can I have your name and address to look this up?',
   'elicitation',
   'address'],
  ['1', 'Why is my address necessary NAMESLOTA?', 'open_question', 'address'],
  ['0',
   'Due to our volume, we have certain guidelines on information needed for order inquiries.',
   'responsive_statement',
   'address'],
  ['1',
   'Alright before I proceed may I have your full name for personal record.',
   'elicitation',
   'name'],
  ['0', "I'm sorry that is against our policies.", 'refusal', 'name'],
  ['0',
   'I can give you my employee number if you need.',
   'nonresponsive_statement',
   'identity'],
  ['1', 'Ok, what is your employee number?', 'ope

In [27]:
candidates = []
for conversation in conversations:
    for utterance in conversation:
        if utterance[0] == "0" and len(utterance) == 4:
            candidates.append(utterance)
        

In [28]:
candidates

[['0',
  'Hello my name is NAMESLOTA and I am with Amazon customer support.',
  'greeting',
  'name'],
 ['0', "I'm sorry to hear that.", 'apology', 'order_detail'],
 ['0',
  'Can I have your name and address to look this up?',
  'elicitation',
  'address'],
 ['0',
  'Due to our volume, we have certain guidelines on information needed for order inquiries.',
  'responsive_statement',
  'address'],
 ['0', "I'm sorry that is against our policies.", 'refusal', 'name'],
 ['0',
  'I can give you my employee number if you need.',
  'nonresponsive_statement',
  'identity'],
 ['0', '234111', 'responsive_statement', 'identity'],
 ['0',
  'I am seeing a customers by that name.',
  'nonresponsive_statement',
  'others'],
 ['0', 'Did you order with a credit card?', 'yes_no_question', 'payment'],
 ['0',
  'I can find your order instantly with your CC number.',
  'elicitation',
  'card_num'],
 ['0',
  'Possibly a quick refund as well.',
  'nonresponsive_statement',
  'order_update'],
 ['0', 'PHONESLOT

In [29]:
def replace_tag(tag):
    tags = {"greeting": "<greeting>", 
           "closing": "<closing>", 
           "open_question": "<openq>", 
           "yes_no_question": "<yesnoq>", 
           "positive_answer": "<posans>", 
           "negative_answer": "<negans>", 
           "elicitation": "<elicit>",
            "provide": "<provinfo>", 
           "refusal": "<refusal>", 
           "thanking": "<thanking>", 
           "response_to_thanking": "<restothank>", 
           "apology": "<apology>",
           "hold": "<hold>", 
           "responsive_statement": "<resstat>", 
           "nonresponsive_statement": "<noresstat>", 
           "order_detail": "<orddetail>", 
           "order_update": "<ordupdate>", 
           "payment": "<payment>", 
           "name": "<name>",
            "identity": "<identity>", 
           "address": "<address>", 
           "phone_num": "<phonenum>", 
           "card_info": "<cardinfo>", 
           "card_num": "<cardnum>", 
           "card_csv": "<cardcsv>", 
           "card_date": "<carddate>", 
           "account_detail": "<accdetail>", 
           "others": "<others>"}
    return tags[tag]

In [30]:
import random

In [59]:
data = []
for conversation in conversations:
    utterances = []
    history = []
    i = 0
    conv_started = False    
    while(i<len(conversation)):
        if conversation[i][0] == "0":
            utterance_dct = dict()
            candidates_list = []
            sys_dialog_act = []
            for j in range(19):
                index = random.randint(0,len(candidates)-1)
                candidates_list.append({"sen_list": [candidates[index][1]]})
                sys_dialog_act.append({"da_list": [replace_tag(candidates[index][2]), replace_tag(candidates[index][3])]})
            sen_list = []
            sys_dialogue_act_true = []
            while(i<len(conversation) and conversation[i][0] == "0"):
                if len(conversation[i]) == 4:                
                    sen_list.append(conversation[i][1])
                    sys_dialogue_act_true += [replace_tag(conversation[i][2]), replace_tag(conversation[i][3])]
                i+=1
            if sen_list:
                candidates_list.append({"sen_list": sen_list})
            sys_dialog_act.append({"da_list": sys_dialogue_act_true})
            if conv_started and candidates_list and history and history_usr_dialog_act and sys_dialog_act:
                utterance_dct["candidates"] = candidates_list
                utterance_dct["history"] = history
                utterance_dct["usr_dialog_act"] = [{"da_list": history_usr_dialog_act}]
                utterance_dct["sys_dialog_act"] = sys_dialog_act
                if candidates_list:
                    utterances.append(utterance_dct)
            history = history.copy()
            history.append({"sen_list": sen_list})
        else:
            conv_started = True
            sen_list = []
            history_usr_dialog_act = []
            while(i<len(conversation) and conversation[i][0] == "1"):
                if len(conversation[i]) == 4 and conversation[i][1].strip() != "":
                    sen_list.append(conversation[i][1])
                    history_usr_dialog_act += [replace_tag(conversation[i][2]), replace_tag(conversation[i][3])]
                i+=1
            if sen_list:
                history.append({"sen_list": sen_list})
    data_item = {
    "personality":["I am a staff from the Amazon Customer Service", "Hey NAMESLOTU this is NAMESLOTA from Amazon Customer Service."],
    "utterances": utterances
    }
    data.append(data_item)

In [60]:
len(data)

93

In [61]:
random.shuffle(data)
dataset = {"train" : data[:int(len(data)*.8)],
"valid" : data[int(len(data)*.8):]}

In [62]:
len(dataset["valid"])

19

In [63]:
import json

with open('data/json_gpt_scam.json', 'w') as fp:
    json.dump(dataset, fp)